## 인터페이스, 추상 베이스 클래스 정의

In [1]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    @abstractmethod
    def write(self, data):
        pass

class SocketStream(IStream):
    def read(self, maxbytes=-1):
        print('reading')
    def write(self, data):
        print('writing')

def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    print('serializing')

if __name__ == '__main__':
    try:
        a = IStream()
    except TypeError as e:
        print(e)

    a = SocketStream()
    a.read()
    a.write('data')

    serialize(None, a)

    import sys

    try:
        serialize(None, sys.stdout)
    except TypeError as e:
        print(e)

    import io
    IStream.register(io.IOBase)

    serialize(None, sys.stdout)

Can't instantiate abstract class IStream with abstract methods read, write
reading
writing
serializing
Expected an IStream
serializing


In [2]:
from abc import ABCMeta, abstractmethod

class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass

    @name.setter
    @abstractmethod
    def name(self, value):
        pass

    @classmethod
    @abstractmethod
    def method1(cls):
        pass

    @staticmethod
    @abstractmethod
    def method2():
        pass


## 커스텀 컨테이너 구현

In [5]:
import collections.abc
import bisect

class SortedItems(collections.Sequence):
    def __init__(self, initial=None):
        self._items = sorted(initial) if initial is not None else []

    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    def add(self, item):
        bisect.insort(self._items, item)

if __name__ == '__main__':
   items = SortedItems([5, 1, 3])
   print(list(items))
   print(items[0])
   print(items[-1])
   items.add(2)
   print(list(items))
   items.add(-10)
   print(list(items))
   print(items[1:4])
   print(3 in items)
   print(len(items))
   for n in items:
       print(n)

[1, 3, 5]
1
5
[1, 2, 3, 5]
[-10, 1, 2, 3, 5]
[1, 2, 3]
True
5
-10
1
2
3
5


## 속성 접근 델리게이팅

In [6]:
class A:
    def spam(self, x):
        print('A.spam')

    def foo(self):
        print('A.foo')

class B:
    def __init__(self):
        self._a = A()   

    def bar(self):
        print('B.bar')

    def __getattr__(self, name):
        return getattr(self._a, name)

if __name__ == '__main__':
    b = B()
    b.bar()
    b.spam(42)

B.bar
A.spam


In [7]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj

    def __getattr__(self, name):
        print('getattr:', name)
        return getattr(self._obj, name)

    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)

    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)

if __name__ == '__main__':
    class Spam:
        def __init__(self, x):
            self.x = x
        def bar(self, y):
            print('Spam.bar:', self.x, y)

    s = Spam(2)

    p = Proxy(s)

    print(p.x)     
    p.bar(3)       
    p.x = 37       

getattr: x
2
getattr: bar
Spam.bar: 2 3
setattr: x 37


In [8]:
class ListLike:
    def __init__(self):
        self._items = []
    def __getattr__(self, name):
        return getattr(self._items, name)

    def __len__(self):
        return len(self._items)
    def __getitem__(self, index):
        return self._items[index]
    def __setitem__(self, index, value):
        self._items[index] = value
    def __delitem__(self, index):
        del self._items[index]

if __name__ == '__main__':
    a = ListLike()
    a.append(2)
    a.insert(0, 1)
    a.sort()
    print(len(a))
    print(a[0])

2
1


## 클래스에 생성자 여러 개 정의

In [10]:
import time

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)

if __name__ == '__main__':
    a = Date(2020, 6, 1)
    b = Date.today()
    print(a.year, a.month, a.day)
    print(b.year, b.month, b.day)

    class NewDate(Date):
        pass

    c = Date.today()
    d = NewDate.today()
    print('Should be Date instance:', Date)
    print('Should be NewDate instance:', NewDate)


2020 6 1
2020 5 30
Should be Date instance: <class '__main__.Date'>
Should be NewDate instance: <class '__main__.NewDate'>


## init 호출 없이 인스턴스 생성

In [12]:
from time import localtime

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def today(cls):
        d = cls.__new__(cls)
        t = localtime()
        d.year = t.tm_year
        d.month = t.tm_mon
        d.day = t.tm_mday
        return d

d = Date.__new__(Date)
print(d)
print(hasattr(d,'year'))

data = { 
    'year' : 2020,
    'month' : 6,
    'day' : 1
}

d.__dict__.update(data)
print(d.year)
print(d.month)

d = Date.today()
print(d.year, d.month, d.day)

False
2020
6
2020 5 30


## 순환 자료 구조에서 메모리 관리 

In [13]:
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

if __name__ == '__main__':
    root = Node('parent')
    c1 = Node('c1')
    c2 = Node('c2')
    root.add_child(c1)
    root.add_child(c2)

    print(c1.parent)
    del root
    print(c1.parent)

Node('parent')
None


In [19]:
class Data:
    def __del__(self):
        print('Data.__del__')

class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []

    def add_child(self, child):
        self.children.append(child)
        child.parent = self

if __name__ == '__main__':
    a = Data()
    del a
    a = Node()
    del a
    a = Node()
    a.add_child(Node())

Data.__del__
Data.__del__


In [20]:
import gc
gc.collect()

Data.__del__
Data.__del__


41

## 비교 연산을 지원하는 클래스 만들기 

In [21]:
from functools import total_ordering
class Room:
    def __init__(self, name, length, width):
        self.name = name
        self.length = length
        self.width = width
        self.square_feet = self.length * self.width

@total_ordering
class House:
    def __init__(self, name, style):
        self.name = name
        self.style = style
        self.rooms = list()

    @property
    def living_space_footage(self):
        return sum(r.square_feet for r in self.rooms)

    def add_room(self, room):
        self.rooms.append(room)

    def __str__(self):
        return '{}: {} square foot {}'.format(self.name, 
                                              self.living_space_footage, 
                                              self.style)

    def __eq__(self, other):
        return self.living_space_footage == other.living_space_footage

    def __lt__(self, other):
        return self.living_space_footage < other.living_space_footage 

h1 = House('h1', 'Cape')
h1.add_room(Room('Master Bedroom', 14, 21))
h1.add_room(Room('Living Room', 18, 20))
h1.add_room(Room('Kitchen', 12, 16))
h1.add_room(Room('Office', 12, 12))

h2 = House('h2', 'Ranch')
h2.add_room(Room('Master Bedroom', 14, 21))
h2.add_room(Room('Living Room', 18, 20))
h2.add_room(Room('Kitchen', 12, 16))

h3 = House('h3', 'Split')
h3.add_room(Room('Master Bedroom', 14, 21))
h3.add_room(Room('Living Room', 18, 20))
h3.add_room(Room('Office', 12, 16))
h3.add_room(Room('Kitchen', 15, 17))
houses = [h1, h2, h3]

print("Is h1 bigger than h2?", h1 > h2) 
print("Is h2 smaller than h3?", h2 < h3) 
print("Is h2 greater than or equal to h1?", h2 >= h1) 
print("Which one is biggest?", max(houses)) 
print("Which is smallest?", min(houses)) 

Is h1 bigger than h2? True
Is h2 smaller than h3? True
Is h2 greater than or equal to h1? False
Which one is biggest? h3: 1101 square foot Split
Which is smallest? h2: 846 square foot Ranch
